In [11]:
import configparser
import numpy as np
import pandas as pd

from os.path import join
from pathlib import Path

In [6]:
scenarios = configparser.ConfigParser()
scenarios.read('scenarios.ini')



['scenarios.ini']

# 小測資

In [9]:
environments = []
for agent in [0, 1]:
    for res in [0, 1]:
        environments.append(f"overbooking_{res}__agent_cancel_{agent}")

In [10]:

for environ in environments:
    for scenario in scenarios.sections():
        scenario_setting = scenarios[scenario]
        performance = pd.DataFrame()
        for instance_id in range(10, 20):
            if instance_id == 14:
                tmp = pd.read_csv(join('history', '0628_small_14', 'solution_up', 'algo_0', 
                                       environ, scenario, '14_performance.csv'))
            else:
                tmp = pd.read_csv(join('history', '0624_small_2', 'solution_up', 'algo_0', 
                                       environ, scenario, f'{instance_id}_performance.csv'))
            performance = pd.concat([performance, tmp], ignore_index=True)
        
        performance.to_csv(join('history', '0624_small_2', 'solution_up', 'algo_0', 
                                environ, scenario, 'performance.csv'))
                


In [17]:
for environ in environments:
    for scenario in scenarios.sections():
        scenario_setting = scenarios[scenario]
        for method in ['algo_0', 'algo_1']:
            performance = pd.DataFrame()
            target_folder = join('history', '0701_small', 'solution_up', method, environ, scenario)
            Path(target_folder).mkdir(exist_ok=True, parents=True)
            for folder in ['0620_small_1', '0624_small_2', '0627_small_3']:
                tmp = pd.read_csv(join('history', folder, 'solution_up', method, 
                                       environ, scenario, 'performance.csv'), index_col=0)
                performance = pd.concat([performance, tmp], ignore_index=True)

            performance.to_csv(join(target_folder, 'performance.csv'))
                

In [2]:
COMPETE_FOLDER_DICT = {
    "relax_gurobi": join('history', 'updated', 'gurobi_solutions', 'up'),  # up_algo_False_relax_True
    "algo": join('history', 'updated', 'algo_solutions', 'up'),  # up_algo_True_relax_False
}




In [19]:
rows = []
for environ in environments:
    for scenario in scenarios.sections():
        scenario_setting = scenarios[scenario]
        for name in COMPETE_FOLDER_DICT:
            gurobi_sol = pd.read_csv(join(
                COMPETE_FOLDER_DICT['relax_gurobi'], 
                environ, 
                scenario, 
                'time_obj.csv'
            )).to_numpy()
            algo_sol = pd.read_csv(join(
                COMPETE_FOLDER_DICT['algo'], 
                environ, 
                scenario, 
                'time_obj.csv'
            )).to_numpy()
            rows.append([
                environ.split('_')[1], environ.split('_')[3],
                scenario_setting['ind_demand_mul'], 
                np.array(scenario_setting['room_rate'][1:-1].split()).argmax(),
                *(algo_sol / gurobi_sol).mean(axis=0),
                gurobi_sol[:, 1].mean(), algo_sol[:, 1].mean()
            ])
            
result = pd.DataFrame(rows, columns=['agent cancel', 'capacity reservation', 
                            'IND demand', 'high room', 'time ratio', 
                            'obj ratio', 'gurobi_obj', 'algo_obj'])
            

In [21]:
result.to_csv('test.csv', index=False)


In [10]:
algo_sol

array([[1.08741010e+00, 2.76813063e+05],
       [1.10361300e+00, 2.06584269e+05],
       [1.05680830e+00, 2.19181851e+05],
       [9.70672900e-01, 1.14627780e+05],
       [1.03697910e+00, 1.83110404e+05]])